In [1]:
import os
import matplotlib.pyplot as plt
import glob
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.mask import mask
from rasterstats import zonal_stats
import fiona
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray
import pandas as pd
import dask

# Set all nan values to np.nan for parameters (only need to run once)

In [ ]:
def reassign_mask_npnan(param_dirs):
    
    for folder in param_dirs:
        files=np.array(sorted(os.listdir(folder)))
        tifs = pd.Series(files).str.contains('.tif')
        files = files[tifs]
        print(folder)
        
        for filename in files:
            with rasterio.open(folder+filename, 'r+') as ds:
                print(filename)
                a = ds.read()# read all raster values
                a[a < 0 ] = np.nan  
                ds.write(a)

In [ ]:
reassign_mask_npnan(all_dirs)

# Create Land Cover Masks

In [ ]:
def create_land_mask(in_dir, out_dir, lc_num):
    files=np.array(sorted(os.listdir(in_dir)))
    
    for filename in files:
        base = os.path.splitext(filename)[0]
        print(base)
        rast = rasterio.open(in_dir+filename)
        meta = rast.meta
        meta.update(dtype = 'float32', nodata = -3.4e+38)
        rast.close()
        
        with rasterio.open(out_dir+base+'_' + str(lc_num) + '.tif', 'w', **meta) as dst:
            with rasterio.open(in_dir+filename) as src:
                data = src.read()
                data = data.astype(np.float32)
                data[data!=lc_num] = np.nan
                data[data==lc_num] = 1
                dst.write(data)

In [ ]:
landcover_in_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Original/'
landcover_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/WoodySavannas_Mask/'
lc_class = 8


In [ ]:
create_land_mask(landcover_in_dir, landcover_out_dir, lc_class)

In [2]:
#ALL DIRECTORIES USED FOR PARAMETER RASTER CALCULATION

grasslands_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Grasslands_Mask/'
croplands_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Croplands_Mask/'
cropnatveg_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/CropNatVeg_Mask/'
savannas_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Savannas_Mask/'
woodysavannas_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/WoodySavannas_Mask/'
evergreenbroad_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/EvergreenBroad_Mask/'
deciduousbroad_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/DeciduousBroad_Mask/'
openshrublands_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/OpenShrublands_Mask/'
closedshrublands_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/ClosedShrublands_Mask/'
barren_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Barren_Mask/'

grasslands_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Grasslands/'
croplands_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Croplands/'
cropnatveg_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_CropNatVeg/'
savannas_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Savannas/' 
woodysavannas_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_WoodySavannas/'
evergreenbroad_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_EvergreenBroad/'
deciduousbroad_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_DeciduousBroad/'
openshrublands_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_OpenShrublands/'
closedshrublands_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_ClosedShrublands/'
barren_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Barren/'

all_dirs = ['/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Resampled/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRTS/Dekads/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Resampled/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Dekads/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Hobbins_ET/Resampled/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Resampled/']

# Multiply a chosen land cover mask by all parameters for each year

In [ ]:
def rastercalc_lcmask(lcmask_dir, param_dirs, out_dir):
    """
    This function iterates through all parameter files and multiplies the land cover mask for a certain year
    to create output rasters of each parameter masked by land cover type
    Note: must change the out name of the land cover in the out file name 
    """
    lcfiles=np.array(sorted(os.listdir(lcmask_dir)))
    
    
    for yr in range(2002,2017):
        lc_yr = [file for file in lcfiles if str(yr) in file]
        for lc in lc_yr:
            lc_mask = rasterio.open(lcmask_dir + lc)
            for folder in param_dirs:
                var_yr = [file for file in sorted(os.listdir(folder)) if str(yr) in file]
                for var in var_yr:
                    var_rast = rasterio.open(folder+var)
                    meta = var_rast.meta
                    var_lcmask = var_rast.read(1)*lc_mask.read(1)
                    with rasterio.open(out_dir + 'evergreenbroad_' + var, 'w', **meta) as dst:
                            dst.write(var_lcmask, 1)
                            print(var)

In [ ]:
rastercalc_lcmask(evergreenbroad_mask_dir, all_dirs, evergreenbroad_masked_out_dir)

# Rasterstats with Admin Boundaries

In [3]:
def zone_stat(raster, band, polygon, stats):
    """
    This function will calculate the zonal stats for each polygon within a raster
    requires gpd_df, raster, object and nodata value
    
    Args: raster = input raster
          band = band of raster
          polygon = polygons to calc zonal stats 
          stats = stat to calculate as string
    """
    
    band = raster.read(band)
    zone_stat = zonal_stats(polygon, band, affine=raster.meta['transform'], nodata = np.nan, stats = stats)
    return zone_stat

In [4]:
def var_poly_join(in_dir, gdf):
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains('.tif')
    files = files[tifs]
    
    for filename in files:
        print(filename)
        raster = rasterio.open(in_dir+filename)
        stats = zone_stat(raster, 1, gdf, 'mean')
        print((list(stats))[-1])
        name = os.path.splitext(os.path.basename(filename))[0]
        gdf['Mean'+ "_" + name] = gpd.GeoDataFrame.from_dict(stats)
    return gdf

In [5]:
lcmasked_in_dir = grasslands_masked_out_dir 

In [ ]:
adminbds = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/AdminBoundaries/Africa_zones_2019/g2008_af_1.shp'
bndry = gpd.read_file(adminbds)

ea_adminbds = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/AdminBoundaries/gadm36_EastAfrica.shp'
ea_bndry = gpd.read_file(ea_adminbds)

oromia = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/AdminBoundaries/Ethiopia/oromia.shp'
oromia_bndry = gpd.read_file(oromia)

In [ ]:
output_shp = var_poly_join(lcmasked_in_dir, ea_bndry)

In [ ]:
output_df = pd.DataFrame(output_shp.drop(columns='geometry'))

In [ ]:
output_df.head()

In [ ]:
output_df.shape

In [ ]:
output_df.to_csv('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Final_TS/'+'ea_grasslands.csv')



# END

# Reassign cloud mask value before calculating stats

In [ ]:
def reassign_masks(in_dir):
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains('.tif')
    files = files[tifs]

    for filename in files:
        with rasterio.open(in_dir+filename, 'r+') as ds:
            print(filename)
            a = ds.read()# read all raster values
            a[a < 0 ] = np.nan  
            ds.write(a)

In [ ]:
reassign_masks(variables_mask_in_dir)

# Trying dask delayed

In [ ]:
@dask.delayed
def read_and_mean(filename, in_dir, admin_gdf):
    """
    input: 
    in_dir - directory of masked land cover type
    filename - a raster variable for particular dekad
    admin_gdf - administrative boundary layer of Africa as geodataframe
    This function opens each raster and extract zonal mean for each variable
    of a particular land cover type
    """
    print(filename)
    raster = rasterio.open(in_dir+filename)
    mean = zone_stat(raster, 1, admin_gdf, 'mean')
    name = os.path.splitext(os.path.basename(filename))[0]
    return {'Mean'+ "_" + name : mean}

In [ ]:
files=np.array(sorted(os.listdir(in_dir)))
tifs = pd.Series(files).str.contains('.tif')
files = files[tifs]

admin_mean_list = []
for filename in files:
    zmeans = read_and_mean(filename, in_dir, bndry)
    admin_mean_list.append(zmeans)


In [ ]:
np.nanx

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
client.compute(admin_mean_list,scheduler = 'processes')

In [ ]:
#var_grass_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Variables_Croplands/'
#year_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Variables_Grasslands/2016/'

In [ ]:
#output_df = output_df[output_df.columns.drop(list(output_df.filter(regex='2014')))] 
#if you want to reverse and remove a specific year from the appended dataframe

# Remove nan pixels from lst

In [ ]:
lst_test = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Clos/Version2/cropland_rs_chirps-v2-Copy1.0.2002.07.1.tif'

In [ ]:
lst_array = lst_test.read(1)

In [ ]:
lst_array.min()

In [ ]:
with rasterio.open(lst_test, 'r+') as ds:
    a = ds.read()# read all raster values
    #lst_array[np.where(lst_array<=0)]
    a[a < 0] = 0  #set all values not cropland as 0
    ds.write(a)

In [ ]:
lst_test2 = rasterio.open(lst_test)
lst_arr = lst_test2.read()
lst_arr.min()

In [ ]:
len(lst_arr[np.where(lst_arr<0)])

In [ ]:
show(lst_test2)

In [ ]:
lst_mask = lst_test.read_masks(1)

In [ ]:
lst_mask

In [ ]:
lst_array.shape

In [ ]:
lst_array[np.where(lst_array<=0)]

In [ ]:
np.where(lst_array<=0)

In [ ]:
with rasterio.open(lc_mask, 'r+') as ds:
    a = ds.read()  # read all raster values
    a[a!=4] = 0  #set all values not cropland as 0
    a[a==4] = 1
    ds.write(a)

In [ ]:
def create_land_mask(in_dir, out_dir, lc_num):
    files=np.array(sorted(os.listdir(in_dir)))
    
    for filename in files:
        base = os.path.splitext(filename)[0]
        print(base)
        with rasterio.open(in_dir+filename, 'r+') as ds:
            rast = ds.read()
            rast[rast!=lc_num] = np.nan
            rast[rast==lc_num] = 1
            meta = ds.meta()
            meta.update(
                dtype='float32')
            ds.write(rast)
                
                
                
#         rast = rasterio.open(in_dir+filename)
#         meta = rast.meta
#         meta.update(
#             dtype='float32')
#         #rast.close()
#         print(meta)
   
        
#         with rasterio.open(out_dir+base+'_' + str(lc_num) + '.tif', 'w', **meta) as dst:
#             #with rasterio.open(in_dir+filename) as src:
#                 data = rast.read()
#                 data[data!=lc_num] = -9999
#                 data[data==lc_num] = 1
#                 dst.write(data)